In [6]:
import sys
import pickle
import numpy as np
import tensorflow as tf
import PIL.Image
import pickle
import os
import re

# Initialize TensorFlow session.
tf.InteractiveSession()
import pandas as pd

TASK_LIST = ['facecat/female','facecat/male',
             'facecat/nosmile', 'facecat/smiles',
             'facecat/old', 'facecat/young',
            'facecat/blond', 'facecat/darkhaired']

In [7]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [8]:
with open('../GANs/karras2018iclr-celebahq-1024x1024.pkl', 'rb') as file:
    G, D, Gs = pickle.load(file)
    # G = Instantaneous snapshot of the generator, mainly useful for resuming a previous training run.
    # D = Instantaneous snapshot of the discriminator, mainly useful for resuming a previous training run.
    # Gs = Long-term average of the generator, yielding higher-quality results than the instantaneous snapshot.

In [9]:
with open('results/stimuli_mapping.obj', 'rb') as file:
    mappings = pickle.load(file)

mappings.drop(['yhat', 'latent'], axis=1, inplace=True)

mappings = mappings.drop_duplicates()

sample_set = mappings.groupby(['task', 'user_id', 'event']).sample(1, random_state=313)

tuples = [tuple(x) for x in sample_set.to_numpy()]

# Generate Results from Brain signal SVR

In [10]:
with open('results/svr_latentsV4.obj', 'rb') as file:
    results_dict = pickle.load(file)

In [11]:
generation_tuple = [] #user id, task, direction, target class, imgid, stepnumber
for img_id, task, user_id, label in tuples:
    
    pos_latents = []
    
    _latents = results_dict[user_id][task][img_id]["POS"]
    
    #pos_latents.append(_latents[1])
    
    _step = np.int(np.floor(len(_latents)/5))
    
    for i in range(0, 4):
        pos_latents.append(_latents[i*_step])
        
    pos_latents.append(_latents[-1])
    
    for number, latent in enumerate(pos_latents):
        generation_tuple.append((latent, user_id, task, "POS", label, img_id, number))
    

' \n    neg_latents = []\n    \n    _latents = results_dict[user_id][task][img_id]["NEG"]\n    \n    #neg_latents.append(_latents[1])\n    \n    _step = np.int(np.floor(len(_latents)/5))\n    \n    for i in range(4):\n        neg_latents.append(_latents[i*_step])\n        \n    neg_latents.append(_latents[-1])\n    \n    for number, latent in enumerate(neg_latents):\n        generation_tuple.append((latent, user_id, task, "NEG", label, img_id, number))\n'

In [12]:
# Generate dummy labels (not used by the official networks).
labels = np.zeros([512] + Gs.input_shapes[1][1:])
sess = tf.Session(config=tf.ConfigProto(
      allow_soft_placement=True, log_device_placement=True))

In [13]:
for latent, user_id, task, direction, label, img_id, number in generation_tuple:
    images = Gs.run(np.array(latent.reshape(1,512)), labels)
    images = np.clip(np.rint((images + 1.0) / 2.0 * 255.0), 0.0, 255.0).astype(np.uint8) # [-1,1] => [0,255]
    images = images.transpose(0, 2, 3, 1) # NCHW => NHWC
    taskname = re.sub('/', '_', task)
    PIL.Image.fromarray(images[0], 'RGB').save(f"results/generated/{user_id}-{taskname}-{label}-{img_id}-{number}-0.jpg")
